# most simple spell correction baseline

In [16]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [95]:
def getNgrams(index):
    df = pd.read_csv(index + ".csv")
    df.columns = ['raw_ngram', 'year', 'match_count', 'volume_count', 'idx', 'n_gram',
       'new_idx', 'is_bastard', 'new_ngram']
    return df

In [ ]:
a = getNgrams('a')

In [21]:
mistakes = a[a.new_ngram == '']
total = len(a)
print('Доля ошибок - ', len(mistakes)/total )

In [ ]:
mistakes[:5]

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [ ]:
Counter(mistakes).most_common(10)

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ выделить ошибочные слова - составить словарь правильных слов и потом сравнивать с ним. 

Мы заранее собрали небольшой корпус текстов с дореволюционной орфографией и немного почистили его удалив отдельную пунктуацию и пунктуацию на границах слов, цифры и латиницу

In [1]:
vocabulary = open('vocabulary.txt', 'r').read()
vocab = []
for line in vocabulary.split("\n"):
    vocab.append(line)

In [2]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
vocab[70:90]

['абажуру',
 'абажуръ',
 'абаза',
 'абазай',
 'абазая',
 'абазеховъ',
 'абазинка',
 'абазинск',
 'абазинскимъ',
 'абазинцевъ',
 'абазъ',
 'абазы',
 'абаи',
 'абаками',
 'абакана',
 'абаканомъ',
 'абаканскимъ',
 'абаканской',
 'абаканску',
 'абаканскъ']

## 1. Проверка на нахождение в словаре

In [3]:
def predict_mistaken(word, vocab):
    '''
    ::input: word, vocabulary
    ::output: 1 or 0
    '''
    return(0 if word in vocab else 1)

### Генерация исправлений

In [25]:
%run VocabMaker.ipynb

Reading a corpus
Estimating cospus size


Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [26]:
corpus = VM.corpus
WORDS = Counter(VM.corpus_words)

In [27]:
WORDS.most_common(10)

[('а', 1),
 ('аа', 1),
 ('ааа', 1),
 ('аааа', 1),
 ('аабб', 1),
 ('аавв', 1),
 ('аагаарду', 1),
 ('аадамауа', 1),
 ('аалбуха', 1),
 ('аалбухъ', 1)]

In [42]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    p = WORDS[word] / N
    return p #if not p==0 else 1/N

In [92]:
P('есмы')

2.8856031487701557e-06

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [47]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмиітьѣѳбю'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [91]:
%%time
correction('пузкин')

CPU times: user 0 ns, sys: 31.2 ms, total: 31.2 ms
Wall time: 18.8 ms


'пушкин'

что происходит в функции edits.

In [76]:
word = 'пузкин'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [77]:
splits[:10]

[('', 'пузкин'),
 ('п', 'узкин'),
 ('пу', 'зкин'),
 ('пуз', 'кин'),
 ('пузк', 'ин'),
 ('пузки', 'н'),
 ('пузкин', '')]

In [78]:
deletes = [L + R[1:] for L, R in splits if R]

In [79]:
deletes[:10]

['узкин', 'пзкин', 'пукин', 'пузин', 'пузкн', 'пузки']

In [80]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [81]:
transposes[:10]

['упзкин', 'пзукин', 'пукзин', 'пузикн', 'пузкни']

In [82]:
letters    = 'йцукенгшщзхъфывапролджэячсмиітьѣѳбюѵ'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [86]:
len(letters)

35

In [84]:
inserts = [L + c + R for L, R in splits for c in letters]

In [85]:
inserts[:10]

['йпузкин',
 'цпузкин',
 'упузкин',
 'кпузкин',
 'епузкин',
 'нпузкин',
 'гпузкин',
 'шпузкин',
 'щпузкин',
 'зпузкин']

Для оценки используем просто долю правильных исправлений.

In [43]:
print(correct/total)

0.860593220338983


Посмотрим какие исправления выбираются для самых частотных опечаток.

In [ ]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [43]:
# в питоне есть библиотеке для нахождения близких строк

In [44]:
from difflib import get_close_matches

In [45]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

CPU times: user 548 ms, sys: 0 ns, total: 548 ms
Wall time: 546 ms


['апофеоз']

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [46]:
import textdistance


In [47]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [48]:
%%time
get_closest_match_with_metric('опофиоз', WORDS, textdistance.hamming)

CPU times: user 868 ms, sys: 4 ms, total: 872 ms
Wall time: 891 ms


('апофеоз', 0.7142857142857143)

In [49]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

CPU times: user 12.2 s, sys: 0 ns, total: 12.2 s
Wall time: 12.1 s


('апофеоз', 0.8571428571428572)

есть что-то побыстрее

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

можно сделать поиск похожих по векторам символов, из которых состоит слово.

In [54]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [55]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [56]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [57]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [ ]:
%%time
get_closest_match_vec('пузкин', X, vec)

В рассмотренных методах при выборе исправления никак не использовался контекст. Про то как это можно сделать, можно почитать в `literature\spellchecking.pdf`, http://www.dialog-21.ru/media/3429/sorokinaashavrinato.pdf